<a href="https://colab.research.google.com/github/rezu98/colab/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
!pip install vaderSentiment
import matplotlib.pyplot as plt
import numpy as np
import nltk
nltk.download('vader_lexicon')
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from openpyxl import Workbook, load_workbook
import re

In [ ]:
!pip install textstat
from textstat import flesch_kincaid_grade, gunning_fog

In [ ]:
file_path = 'results.xlsx'

In [ ]:
### Sentiment Intensity Analyzer
# Example text
sample_text = "The Monetary Policy Board of the Bank of Korea decided today to leave the Base Rate unchanged at 3.50% for the intermeeting period. Although inflation is projected to continue its underlying trend of a slowdown, uncertainties regarding the future path of inflation and growth have risen significantly due to a prolongation of restrictive monetary policy stances in major countries and heightened geopolitical risks. In addition, it is forecast that the pace of inflation slowdown will moderate more than previously expected, and it is necessary to monitor household debt growth. The Board, therefore, sees that it is appropriate to maintain its current restrictive policy stance. Regarding the need to raise the Base Rate further, the Board will make a judgement while assessing the changes in domestic and external policy conditions. The currently available information suggests that uncertainties regarding economic and inflationary trends have increased across the global economy, driven by a prolongation of restrictive monetary policy stances in major countries and by the Israel-Hamas conflict. Global economic growth is projected to continue slowing. Inflation in major countries still remains high, though falling gradually, and upside risks have increased due to the rise in global oil prices. In global financial markets, volatility of major price variables has increased with government bond yields rising significantly and with the U.S. dollar strengthening considerably. Looking ahead, the Board sees global economic growth and global financial markets as likely to be affected by the movements of global oil prices and the global inflation slowdown, by monetary policy changes in major countries and their effects, and by developments in the Israel-Hamas conflict. Domestic economic growth has continued to improve at a modest pace owing to the easing of sluggishness in exports, although the recovery in private consumption has been somewhat slow. Labor market conditions have been generally favorable, as both a low unemployment rate and a robust increase in the number of persons employed have continued. Going forward, domestic economic growth is expected to improve gradually with the easing of the sluggishness in exports. GDP growth for the year is expected to be generally consistent with the August forecast of 1.4%. However, uncertainties surrounding the economic outlook are judged to be elevated, affected by heightened geopolitical risks and by the prolongation of restrictive monetary policy stances in major countries. Consumer price inflation has risen from August to 3.7% in September, due to the increase in the price of energy and of agricultural products. However, both core inflation (excluding changes in food and energy prices from the CPI) and short-term inflation expectations among the general public have stayed at 3.3% in September, the same as in August. Looking ahead, it is forecast that consumer price inflation will fall to the lower-3% range at the end of this year and will continue to gradually moderate in 2024. However, upside risks to inflation have increased due to the effects of higher global oil prices and exchange rates, and due to the Israel-Hamas conflict. Accordingly, it is judged that the timing of consumer price inflation converging on the target level is more likely to be delayed than previously expected. Meanwhile, core inflation is also projected to maintain its underlying slowing trend, owing to the weakening of demand-side pressures. However, the pace of the slowdown is likely to be more modest than previously forecast due to the continuing spillover effects of accumulated cost pressure. In financial and foreign exchange markets, volatility has increased as the U.S. Federal Reserve has signaled a prolongation of a high policy rate and as geopolitical risks have expanded. Long-term Korean Treasury bond yields and the Korean won to U.S. dollar exchange rate have risen significantly and stock prices have fallen. Meanwhile, the risks to some non-bank financial sectors have eased. Housing prices have continued their upward trend, especially in Seoul and its surrounding areas. Household loans have continued to increase, mainly driven by housing-related loans. The Board will continue to conduct monetary policy in order to stabilize consumer price inflation at the target level over the medium-term horizon as it monitors economic growth, while paying attention to financial stability. While domestic economic growth is forecast to gradually improve, uncertainties surrounding the policy decision have also risen. The Board, therefore, will maintain a restrictive policy stance for a considerable time with an emphasis on ensuring price stability, while making a judgement regarding the need to raise the Base Rate further. In this process, the Board will thoroughly assess the inflation slowdown, financial stability risks, economic downside risks, monetary policy changes in major countries, household debt growth, and developments in geopolitical risks."
# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()
# Perform sentiment analysis
scores = analyzer.polarity_scores(sample_text)
print(scores)

In [ ]:
### Sentiment Analyze
### by loughran and mcdonald(2019)

import datetime as dt
import sys
import nltk
nltk.download('punkt')

## loading master dictionary

def load_masterdictionary(file_path, print_flag=False, f_log=None, get_other=False):
    start_local = dt.datetime.now()
    # Setup dictionaries
    _master_dictionary = {}

    # Loop through words and load dictionaries
    with open(file_path, encoding="utf-8") as f:
        _total_documents = 0
        _md_header = f.readline()  # Consume header line
        print()
        for line in f:
            cols = line.rstrip('\n').split(',')
            word = cols[0].upper()  # Convert word to uppercase
            negative_value = int(cols[7])
            positive_value = int(cols[8])
            uncertainty_value = int(cols[9])
            _master_dictionary[word] = {'negative': negative_value, 'positive': positive_value, 'uncertainty': uncertainty_value}

    if print_flag:
        print('\r', end='')  # clear line
        print(f'\nMaster Dictionary loaded from file:\n  {file_path}\n')
        print(f'  master_dictionary has {len(_master_dictionary):,} words.\n')

    if get_other:
        return _master_dictionary, _md_header, _total_documents
    else:
        return _master_dictionary

In [ ]:
## evaluate sentiment indices

def evaluate_sentiment_indices(text, master_dictionary):
    words = nltk.word_tokenize(text)  # Tokenize the text into words
    words_uppercase = [word.upper() for word in words]  # Convert all words to uppercase
    total_words_count, negative_score, positive_score, uncertain_words_count = 0, 0, 0, 0
    negative_words_count, positive_words_count = 0, 0
    for word in words_uppercase:
        if word in master_dictionary:
            total_words_count += 1  # Increment the total word count
            if master_dictionary[word]['uncertainty'] != 0:
                uncertain_words_count += 1
            if master_dictionary[word]['negative'] > 0:
                negative_words_count += 1
            if master_dictionary[word]['positive'] > 0:
                positive_words_count += 1

    negative_index = negative_words_count / total_words_count  # Calculate the negative index
    positive_index = positive_words_count / total_words_count  # Calculate the positive index
    uncertainty_index = uncertain_words_count / total_words_count  # Calculate the uncertainty index

    return negative_index, positive_index, uncertainty_index, total_words_count, uncertain_words_count, negative_words_count, positive_words_count

if __name__ == '__main__':
    start = dt.datetime.now()
#    print(f'\n\n{start.strftime("%c")}\nPROGRAM NAME: {sys.argv[0]}\n')
    md = r'c.csv'  # Update with your file path
    master_dictionary = load_masterdictionary(md, True)

In [ ]:
def evaluate_readability(text):
    sentences = nltk.sent_tokenize(text)
    words = nltk.word_tokenize(text)
    num_sentences = len(sentences)
    num_words = len(words)
    num_syllables = 0
    for word in words:
        num_syllables += textstat.syllable_count(word)
    return textstat.flesch_kincaid_grade(text), textstat.gunning_fog(text)

In [ ]:
# File path for the Excel file
file_path = 'results.xlsx'

In [ ]:
# Example FOMC statement to evaluate

date = "2010.10"

fomc_statement = """
   The Monetary Policy Committee of the Bank of Korea decided today to maintain the Base Rate at its current level (2.25%) for the intermeeting period.



In the global economy, emerging market economies have sustained their favorable performance, and the economies of major advanced countries have largely continued their moderate recovery trend, even though the pace of the recovery in the US economy has slowed somewhat. Looking ahead, there exists the possibility of the heightened volatility of economic activity and exchange rates in major countries acting as a risk factor for the global economy.



The underlying upward trend of domestic business activity has been maintained. Even though consumption has faltered, exports have sustained their buoyancy and facilities investment has increased. And, led by the private sector, labor market conditions have shown an improving trend.



The domestic economy is expected to continue on an underlying upward track, even in the presence of external risk.



Consumer price inflation has increased due to a sudden rise in farm product prices, and in the future, upward pressures on the demand side are expected to continue, being associated with the continued upswing in activity. In the real estate market, housing sales prices have continued to decline in Seoul and its surrounding areas, while those in other areas have maintained their increase.


In the financial markets, stock prices have risen, the Korean won has appreciated, and market interest rates have declined in response chiefly to the expanded inflow of foreign portfolio investment funds. The scale of the growth in mortgage lending has widened due to the increase in the numbers of newly occupied apartments, even though house transactions have been inactive.



Looking ahead, the Committee will conduct monetary policy in such a way as to help the economy maintain price stability, while sustaining sound growth under the accommodative policy stance. In carrying out policy, it will take overall account of financial and economic conditions at home and abroad



 """

 ## run!!!

if __name__ == '__main__':
    start = dt.datetime.now()
    md = r'c.csv'  # Update with your file path
    master_dictionary = load_masterdictionary(md, True)

    # Remove unnecessary line breaks and spaces
    processed_text = re.sub(r'\n+', ' ', fomc_statement)  # Replace multiple line breaks with a single space
    processed_text = re.sub(r' +', ' ', processed_text)  # Replace multiple spaces with a single space

    negative_index, positive_index, uncertainty_index, total_words_count, uncertain_words_count, negative_words_count, positive_words_count = evaluate_sentiment_indices(processed_text, master_dictionary)
    flesch_kincaid_grade, gunning_fog_index = evaluate_readability(processed_text)

    print(f"\nNumber of uncertain words: {uncertain_words_count}")
    print(f"Number of negative words: {negative_words_count}")
    print(f"Number of positive words: {positive_words_count}")
    print(f"Number of total words: {total_words_count}")
    print(f"Negative index for the FOMC statement: {negative_index:.2f}")
    print(f"Positive index for the FOMC statement: {positive_index:.2f}")
    print(f"Uncertainty index for the FOMC statement: {uncertainty_index:.2f}")
    print(f"Flesch-Kincaid Grade Level: {flesch_kincaid_grade}")
    print(f"Gunning Fog Index: {gunning_fog_index}")

    # Printing the actual words defined as positive, negative, and uncertainty
    words = nltk.word_tokenize(processed_text)
    words_uppercase = [word.upper() for word in words]  # Convert all words to uppercase
    words_with_sentiment = [(word, master_dictionary[word]) for word in words_uppercase if word in master_dictionary]
    positive_words = [word for word, sentiment in words_with_sentiment if sentiment['positive'] > 0]
    negative_words = [word for word, sentiment in words_with_sentiment if sentiment['negative'] > 0]
    uncertain_words = [word for word, sentiment in words_with_sentiment if sentiment['uncertainty'] > 0]
    print(f"\nPositive words: {positive_words}")
    print(f"Negative words: {negative_words}")
    print(f"Uncertainty words: {uncertain_words}")

# Check if the file already exists, and load it if it does
try:
    workbook = load_workbook(file_path)
    sheet = workbook.active
except FileNotFoundError:
    workbook = Workbook()
    sheet = workbook.active
    sheet.append(['Date', 'FOMC Statement', 'Positive Words', 'Negative Words', 'Uncertainty Words', 'Total Words', 'Positive Words List', 'Negative Words List', 'Uncertainty Words List', 'Flesch-Kincaid Grade Level', 'Gunning Fog Index'])

# Append results to the Excel file
sheet.append([date, processed_text, positive_words_count, negative_words_count, uncertain_words_count, total_words_count, ', '.join(positive_words), ', '.join(negative_words), ', '.join(uncertain_words), flesch_kincaid_grade, gunning_fog_index])

# Save the workbook
workbook.save(file_path)


In [ ]:
## Visualization of tokenized words by categories

categories = {'Positive': positive_words, 'Negative': negative_words, 'Uncertainty': uncertain_words}
for category, words in categories.items():
    word_counts = {word: words.count(word) for word in set(words)}
    sorted_word_counts = dict(sorted(word_counts.items(), key=lambda item: item[1], reverse=True))
    plt.figure(figsize=(10, 6))
    plt.barh(list(sorted_word_counts.keys()), list(sorted_word_counts.values()), color='skyblue')
    plt.ylabel('Words')
    plt.xlabel('Frequency')
    plt.title(f'Word Frequency for {category} Words')
    plt.show()

!!! Web crawling of NAVER

In [128]:
!pip install selenium
!pip install bs4
!pip install webdriver_manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=762ba4d004870e8c4f1a5b60aed64600bbf11c75e199e3966a554d7da871c62c
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [143]:
from selenium.webdriver.chrome.service import Service

In [132]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

#웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

In [141]:
chrome_options = webdriver.ChromeOptions()

In [133]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)


# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search,start_pg,end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ",url)
        return url
    else:
        urls= []
        for i in range(start_pg,end_pg+1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ",urls)
        return urls

In [134]:
##########뉴스크롤링 시작###################

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")

#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")

# naver url 생성
search_urls = makeUrl(search,page,page2)

검색할 키워드를 입력해주세요:한국은행

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):100

크롤링할 종료 페이지:  100 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=한국은행&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=한국은행&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=한국은행&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=한국은행&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=한국은행&start=41', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=한국은행&start=51', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=한국은행&start=61', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=한국은행&start=71', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=한국은행&start=81', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=한국은행&start=91', 'https://search.naver.c

In [136]:
## selenium으로 naver news만 뽑아오기##
# 버전에 상관 없이 os에 설치된 크롬 브라우저 사용
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(3)


# selenium으로 검색 페이지 불러오기 #

naver_urls=[]

for i in search_urls:
    driver.get(i)
    time.sleep(1) #대기시간 변경 가능

    # 네이버 기사 눌러서 제목 및 본문 가져오기#
    # 네이버 기사가 있는 기사 css selector 모아오기
    a = driver.find_elements(By.CSS_SELECTOR,'a.info')

    # 위에서 생성한 css selector list 하나씩 클릭하여 본문 url얻기
    for i in a:
        i.click()

        # 현재탭에 접근
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(3) #대기시간 변경 가능

        # 네이버 뉴스 url만 가져오기

        url = driver.current_url
        print(url)

        if "news.naver.com" in url:
            naver_urls.append(url)

        else:
            pass

        # 현재 탭 닫기
        driver.close()

        # 다시처음 탭으로 돌아가기(매우 중요!!!)
        driver.switch_to_window(driver.window_handles[0])

print(naver_urls)

AttributeError: ignored

In [144]:
## selenium으로 navernews만 뽑아오기##
# 버전에 상관 없이 os에 설치된 크롬 브라우저 사용
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

AttributeError: ignored

In [ ]:
###naver 기사 본문 및 제목 가져오기###

# ConnectionError방지
headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102" }

titles = []
contents=[]
for i in naver_urls:
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    #print(html)

    #뉴스 제목 가져오기
    title = html.select("div.content > div.article_header > div.article_info > h3")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1,repl='',string=title)
    titles.append(title)

    #뉴스 본문 가져오기

    content = html.select("div.content > div#articleBody > div#articleBodyContents")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    #html태그제거 및 텍스트 다듬기
    content = re.sub(pattern=pattern1,repl='',string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2,'')

    contents.append(content)

print(titles)
print(contents)

In [ ]:
### force delete

import os
import shutil
shutil.rmtree('zipf.png', ignore_errors=True)